In [1]:
import numpy as np 
import pandas as pd 

In [2]:
import os 
inputFolder = '../Research Work/Docs/' 
for root, directories, filenames in os.walk(inputFolder): 
    for filename in filenames: 
        print(os.path.join(root,filename))
        with open(os.path.join(root, filename)) as f:
            line_count = 0
            id_set = set()
            for l in f.readlines():
                line_count += 1
                if filename == "CISI.REL":
                    id_set.add(l.lstrip(" ").split(" ")[0])
                elif l.startswith(".I "):
                    id_set.add(l.split(" ")[1].strip())
            print(f"{filename} : {len(id_set)} items, over {line_count} lines.")

../Research Work/Docs/CISI.ALL
CISI.ALL : 1460 items, over 108747 lines.
../Research Work/Docs/CISI.BLN
CISI.BLN : 0 items, over 86 lines.
../Research Work/Docs/CISI.QRY
CISI.QRY : 112 items, over 1525 lines.
../Research Work/Docs/CISI.REL
CISI.REL : 76 items, over 3114 lines.


In [3]:
with open('../Research Work/Docs/CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
# print n lines
n = 5
for l in lines[:n]:
    print(l)

.I 1
.T 18 Editions of the Dewey Decimal Classifications
.A Comaromi, J.P.
.W The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad.
.X 1	5	1 92	1	1 262	1	1 556	1	1 1004	1	1 1024	1	1 1024	1	1


In [4]:
doc_set = {}
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

# Print something to see the dictionary structure, etc.
print(f"Number of documents = {len(doc_set)}" + ".\n")
print(doc_set["3"]) # note that the dictionary indexes are strings, not numbers.

Number of documents = 1460.

Two Kinds of Power An Essay on Bibliographic Control Wilson, P. The relationships between the organization and control of writings and the organization and control of knowledge and information will inevitably enter our story, for writings contain, along with much else, a great deal of mankind's stock of knowledge and information.  Bibliographical control is a form of power, and if knowledge itself is a form of power, as the familiar slogan claims, bibliographical control is in a certain sense power over power, power to obtain the knowledge recorded in written form.  As writings are not simply, and not in any simple way, storehouses of knowledge, we cannot satisfactorily discuss bibliographical control as simply control over the knowledge and information contained in writings. 


In [5]:
with open('../Research Work/Docs/CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = l.split(" ")[1].strip()
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""
        
# Print something to see the dictionary structure, etc.
print(f"Number of queries = {len(qry_set)}" + ".\n")
print(qry_set["3"]) # note that the dictionary indexes are strings, not numbers. 

Number of queries = 112.

What is information science?  Give definitions where possible.


In [6]:
rel_set = {}
with open('../Research Work/Docs/CISI.REL') as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]
        doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)
        if qry_id == "7":
            print(l.strip("\n"))
    
# Print something to see the dictionary structure, etc.
print(f"\nNumber of mappings = {len(rel_set)}" + ".\n")
print(rel_set["3"]) # note that the dictionary indexes are strings, not numbers. 

     7    310	0	0.000000
     7    320	0	0.000000
     7    332	0	0.000000
     7    375	0	0.000000
     7    376	0	0.000000
     7    645	0	0.000000
     7    724	0	0.000000
     7    725	0	0.000000

Number of mappings = 76.

['60', '85', '114', '123', '126', '131', '133', '136', '138', '140', '346', '359', '363', '372', '412', '445', '454', '461', '463', '469', '532', '537', '540', '553', '554', '555', '585', '590', '599', '640', '660', '664', '803', '901', '909', '911', '1027', '1053', '1169', '1179', '1181', '1190', '1191', '1326']


In [7]:
list1 = list(doc_set.values())

In [63]:
qry_set
queries = list(qry_set.values())
len(queries)

112

In [9]:
list1 = list1+queries
len(list1)

1572

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem import WordNetLemmatizer

def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def lemma(data):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + lemmatizer.lemmatize(w)
    return new_text

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = lemma(data)
    data = remove_punctuation(data)
    data = lemma(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    #data = word_tokenize(data)
    return data

In [11]:
for i, sentence in enumerate(list1):
	tokenized= []
	tokenized.append(preprocess(sentence))
	list1[i] = tokenized

In [12]:
for i in range(len(list1)):
    for sentence in list1[i]:
        tokenized= []
        tokenized.append(word_tokenize(sentence))
        list1[i] = tokenized

In [13]:
output = []
for sublist in list1:
    for item in sublist:
        output.append(item)

In [14]:
from gensim.models import word2vec

In [15]:
model = word2vec.Word2Vec(output, size=100, window=8, min_count=1, workers=4) 

In [212]:
qry_set
queries = list(qry_set.values())
len(queries)

112

In [213]:
sentence = queries[25]
arr2 = []
arr2.append(sentence)

for i, sentence in enumerate(arr2):
	tokenized= []
	tokenized.append(preprocess(sentence))
	arr2[i] = tokenized
    
for i in range(0, len(arr2)):
    for sentence in arr2[i]:
        tokenized= []
        tokenized.append(word_tokenize(sentence))
        arr2[i] = tokenized
        
output2 = []
for sublist in arr2:
    for item in sublist:
        output2.append(item)
        
output3 = []
for sublist in output2:
    for item in sublist:
        output3.append(item) 
        
output3

['cost',
 'determination',
 'cost',
 'associated',
 'system',
 'automated',
 'information']

In [214]:
queries = []
#queries = model.most_similar(positive=['dewey','decimal'], topn = 10)
queries = [item[0] for item in model.most_similar(positive=output3, topn = 5)]
for i in range(0, len(output3)):
    queries.append(output3[i])
queries

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


['design',
 'evaluation',
 'dissemination',
 'line',
 'discussed',
 'cost',
 'determination',
 'cost',
 'associated',
 'system',
 'automated',
 'information']

In [215]:
def doc_ranking(query):
    rank = []
    for x in range(0,1460):
        intersection = set(query).intersection(set(output[x]))
        union = set(query).union(set(output[x]))
        rank.append(len(intersection)/len(union))
    return rank

In [216]:
text=[]
text= doc_ranking(queries)

In [217]:
text

[0.0196078431372549,
 0.014492753623188406,
 0.02,
 0.03333333333333333,
 0.0,
 0.02608695652173913,
 0.0196078431372549,
 0.01,
 0.0,
 0.025,
 0.017543859649122806,
 0.020202020202020204,
 0.0196078431372549,
 0.0,
 0.02127659574468085,
 0.015503875968992248,
 0.025252525252525252,
 0.0547945205479452,
 0.0,
 0.0,
 0.0,
 0.0,
 0.018518518518518517,
 0.0,
 0.009615384615384616,
 0.017857142857142856,
 0.08196721311475409,
 0.019230769230769232,
 0.023255813953488372,
 0.01818181818181818,
 0.0,
 0.014925373134328358,
 0.023809523809523808,
 0.01639344262295082,
 0.0,
 0.0,
 0.021739130434782608,
 0.0,
 0.0,
 0.018867924528301886,
 0.05555555555555555,
 0.024390243902439025,
 0.01694915254237288,
 0.015151515151515152,
 0.0,
 0.02531645569620253,
 0.020618556701030927,
 0.015151515151515152,
 0.0375,
 0.023809523809523808,
 0.0,
 0.0,
 0.043478260869565216,
 0.06060606060606061,
 0.0,
 0.02631578947368421,
 0.05128205128205128,
 0.015151515151515152,
 0.08163265306122448,
 0.03225806451

In [218]:
N=200
rel=[]
res = sorted(range(len(text)), key = lambda sub: text[sub])[-N:] 
for x in range(0,200):
    if(res[x] < 1460):
        rel.append(res[x]+1)
print("Indices of Top 100 Most Relevant Documents : " + str(rel)) 
print(len(rel))

Indices of Top 100 Most Relevant Documents : [137, 325, 434, 1104, 1209, 594, 1366, 321, 288, 796, 807, 829, 851, 986, 993, 1132, 1296, 607, 625, 136, 334, 451, 848, 868, 925, 1155, 1376, 134, 406, 534, 801, 849, 74, 267, 299, 870, 979, 1377, 621, 866, 948, 57, 332, 467, 535, 631, 647, 865, 597, 895, 1375, 222, 318, 454, 501, 508, 529, 537, 744, 1035, 1089, 1293, 572, 382, 449, 474, 726, 1078, 1143, 514, 826, 840, 553, 645, 1448, 18, 575, 41, 67, 177, 483, 526, 642, 695, 1196, 1223, 694, 822, 1368, 452, 1092, 486, 1437, 723, 846, 175, 599, 707, 1084, 1093, 1096, 1136, 1447, 490, 224, 648, 54, 281, 523, 539, 671, 1022, 1364, 465, 120, 639, 889, 450, 630, 704, 710, 1171, 66, 1057, 135, 717, 1112, 114, 179, 727, 1305, 1361, 1410, 502, 703, 731, 853, 897, 1125, 1367, 1413, 699, 528, 990, 1326, 468, 629, 1053, 1110, 1170, 1179, 482, 515, 294, 481, 157, 466, 513, 532, 875, 1297, 611, 1126, 1358, 1405, 620, 1114, 1283, 1307, 1121, 59, 615, 654, 27, 839, 780, 433, 446, 1264, 1282, 1303, 690, 8

In [219]:
print(rel_set["26"])

['17', '27', '36', '67', '74', '83', '126', '158', '164', '192', '214', '218', '307', '338', '364', '365', '372', '381', '446', '458', '465', '468', '474', '490', '491', '495', '496', '497', '500', '507', '523', '534', '584', '591', '615', '620', '639', '675', '690', '723', '732', '779', '813', '822', '848', '849', '860', '865', '979', '1100', '1264', '1366', '1368', '1374', '1376', '1377']


In [220]:
rel1= rel_set["26"]
for i in range(0, len(rel1)): 
    rel1[i] = int(rel1[i])
    
print(rel1)

[17, 27, 36, 67, 74, 83, 126, 158, 164, 192, 214, 218, 307, 338, 364, 365, 372, 381, 446, 458, 465, 468, 474, 490, 491, 495, 496, 497, 500, 507, 523, 534, 584, 591, 615, 620, 639, 675, 690, 723, 732, 779, 813, 822, 848, 849, 860, 865, 979, 1100, 1264, 1366, 1368, 1374, 1376, 1377]


In [221]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

In [222]:
x = len(intersection(res, rel1))
y = len(rel1)

In [223]:
x

14

In [224]:
y

56

In [225]:
x/y * 100

25.0